In [2]:
# import libraries

%matplotlib inline
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Input, InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
import keras
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler
from scipy.linalg import eigh
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

Load dataset


In [4]:
mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


Preprocessing dataset

In [5]:
# Normalization

X_train = X_train / 255.
X_test = X_test / 225.

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

In [6]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(60000, 784) (60000,) (10000, 784) (10000,)


PCA

In [7]:
# may not need to use StandardScaler. Instead, could just subtract the average from each data point

standardized_scalar = StandardScaler()

standardized_data = standardized_scalar.fit_transform(X_train)
standardized_data.shape

standardized_data_test = standardized_scalar.transform(X_test)
standardized_data_test.shape

(10000, 784)

In [9]:
# covariance matrix is X_transpose times X
cov_matrix = np.matmul(standardized_data.T, standardized_data)


conv_matrix_test = np.matmul(standardized_data_test.T, standardized_data_test)

print(cov_matrix.shape)


(784, 784)


In [10]:
# calculate eigenvalues and eigenvectors for train set

lambdas, vectors = eigh(cov_matrix, eigvals=(0, 299))

print(vectors.shape)

vectors = vectors.T
vectors.shape

(784, 300)


(300, 784)

In [11]:
# Aim: create a diagonal matrix -- so that we could maximize variance values
# Choose a matrix P in which every row is an eigenvector of the covariance matrix.
# This way we could create the diagonal matrix with P(X_T*X)

new_coordinates_train = np.matmul(vectors, standardized_data.T).T

new_coordinates_test = np.matmul(vectors, standardized_data_test.T).T

In [12]:
new_coordinates_train.shape, new_coordinates_test.shape

((60000, 300), (10000, 300))

In [13]:
X_train = new_coordinates_train
print(X_train.shape, y_train.shape)

(60000, 300) (60000,)


In [14]:
X_test = new_coordinates_test
print(X_test.shape, y_test.shape)

(10000, 300) (10000,)


In [15]:
nclasses = y_train.max() - y_train.min() + 1
y_train = to_categorical(y_train, num_classes = nclasses)
y_test = to_categorical(y_test, num_classes = nclasses)

print("Shape of ytrain after encoding: ", y_train.shape)
print("Shape of ytest after encoding: ", y_test.shape)

Shape of ytrain after encoding:  (60000, 10)
Shape of ytest after encoding:  (10000, 10)


Build model

In [18]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape = (X_train.shape[1],)))
# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

Train model

In [19]:
callback = tf.keras.callbacks.EarlyStopping(monitor='acc', patience=3)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs = 100, batch_size = 64, callbacks = callback, shuffle=True, validation_data = (X_test, y_test))

Epoch 1/100
938/938 [==============================] - 2s 2ms/step - loss: 1.8688 - acc: 0.4234 - val_loss: 1.3661 - val_acc: 0.6053
Epoch 2/100
938/938 [==============================] - 1s 1ms/step - loss: 1.1871 - acc: 0.6575 - val_loss: 1.0192 - val_acc: 0.7003
Epoch 3/100
938/938 [==============================] - 1s 1ms/step - loss: 0.9120 - acc: 0.7347 - val_loss: 0.8775 - val_acc: 0.7383
Epoch 4/100
938/938 [==============================] - 1s 1ms/step - loss: 0.7611 - acc: 0.7750 - val_loss: 0.7979 - val_acc: 0.7539
Epoch 5/100
938/938 [==============================] - 1s 1ms/step - loss: 0.6684 - acc: 0.7992 - val_loss: 0.7676 - val_acc: 0.7659
Epoch 6/100
938/938 [==============================] - 1s 1ms/step - loss: 0.6032 - acc: 0.8173 - val_loss: 0.7397 - val_acc: 0.7743
Epoch 7/100
938/938 [==============================] - 1s 1ms/step - loss: 0.5553 - acc: 0.8309 - val_loss: 0.7237 - val_acc: 0.7812
Epoch 8/100
938/938 [==============================] - 1s 1ms/step - 

Use NN for classifying MNIST

In [21]:
# Load dataset
mnist = tf.keras.datasets.mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

# Preoprocessing data
train_X = train_X /255.
test_X = test_X /255.

# Categorical label
nclasses = train_y.max() - train_y.min() + 1
train_y = to_categorical(train_y, num_classes = nclasses)
test_y = to_categorical(test_y, num_classes = nclasses)

# Build model
nn_model = Sequential()
nn_model.add(Flatten(input_shape=(train_X.shape[1], train_X.shape[2])))
nn_model.add(Dense(512, activation='relu'))
nn_model.add(Dense(256, activation='relu'))
nn_model.add(Dense(128, activation='relu'))
nn_model.add(Dense(10, activation='softmax'))

# Train model
callback = tf.keras.callbacks.EarlyStopping(monitor='acc', patience=3)
nn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
nn_model.fit(train_X, train_y, epochs = 100, batch_size = 64, callbacks = callback, shuffle=True, validation_data = (test_X, test_y))

Epoch 1/100
938/938 [==============================] - 6s 6ms/step - loss: 0.2037 - acc: 0.9406 - val_loss: 0.0961 - val_acc: 0.9689
Epoch 2/100
938/938 [==============================] - 5s 5ms/step - loss: 0.0828 - acc: 0.9743 - val_loss: 0.0831 - val_acc: 0.9754
Epoch 3/100
938/938 [==============================] - 5s 5ms/step - loss: 0.0571 - acc: 0.9819 - val_loss: 0.0799 - val_acc: 0.9755
Epoch 4/100
938/938 [==============================] - 5s 5ms/step - loss: 0.0434 - acc: 0.9858 - val_loss: 0.0717 - val_acc: 0.9799
Epoch 5/100
938/938 [==============================] - 5s 5ms/step - loss: 0.0351 - acc: 0.9889 - val_loss: 0.0822 - val_acc: 0.9771
Epoch 6/100
938/938 [==============================] - 5s 5ms/step - loss: 0.0290 - acc: 0.9906 - val_loss: 0.0828 - val_acc: 0.9777
Epoch 7/100
938/938 [==============================] - 5s 6ms/step - loss: 0.0263 - acc: 0.9917 - val_loss: 0.0770 - val_acc: 0.9813
Epoch 8/100
938/938 [==============================] - 5s 5ms/step - 